In [ ]:
import sys
sys.path.append("mrcnn")
from m_rcnn import *
import cv2
from visualize import *
import requests
import json
import numpy as np
from PIL import Image
import io
import base64
import warnings
from flask import Flask, request, Response
from flask import send_file
import imutils
import glob
import tensorflow as tf
import keras
from keras import backend as K
colors = random_colors(80)


colors = random_colors(80)
net = cv2.dnn.readNet("yolov3_training_final.weights","yolov3_testing.cfg")
classes = ['osteophytes']
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]# get the output layers which are 82,94,106 indexes
# Loading image
font = cv2.FONT_HERSHEY_PLAIN

warnings.filterwarnings('ignore')
def encode(val):
    val=val.replace("+","BGABG")
    val=val.replace("=","AGAAGA")
    val=val.replace("/","OPAWQQQQ")
    return val
def decode(val):
    val=val.replace("BGABG","+")
    val=val.replace("AGAAGA","=")
    val=val.replace("OPAWQQQQ","/")
    return val
def stringToImage(base64_string):
    imgdata = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(imgdata))
def toRGB(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)

def get_yolo_encoded(img):
    trans=np.zeros_like(img)
    height,width,c=img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outputs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.7, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            cv2.rectangle(trans, (x, y), (x + w, y + h), (0,0,255), 1)
    
    _,buffer_yolo=cv2.imencode('.jpg',trans)
    img_encoded_yolo=base64.b64encode(buffer_yolo)
    yoloy_st=encode((str(img_encoded_yolo)[2:])[:-1])
    return yoloy_st
    
app=Flask(__name__)

@app.route('/')
def index():
    return "Knee instance segmentation"

@app.route('/knee')
def get_json(methods=["GET"]):
    sess = K.get_session()
    test_model, inference_config =load_inference_model(1,"mask_rcnn_object_0005.h5")
    model_2, inference_config_2 =load_inference_model(1,"knee_space_narrowing.h5")

    input_url=request.args.get('image',type=str)
    img_string=decode(input_url)
    img = toRGB(stringToImage(img_string))
    transparent=np.zeros_like(img)
    img_zeros=np.zeros_like(img)

    height,width,c=img.shape
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,c=img.shape
    r = test_model.detect(np.array(image).reshape(-1,h,w,c))[0]
    r_2 = model_2.detect(np.array(image).reshape(-1,h,w,c))[0]
    colors = random_colors(80)
    object_count = len(r["class_ids"])
    for i in range(object_count):
        mask = r["masks"][:, :, i]
        contours = get_mask_contours(mask)        
        for cnt in contours:
            cv2.polylines(img, [cnt], True, colors[i], 2)#only the outlines
            cv2.polylines(transparent, [cnt], True, colors[i], 1)#only the outlines
    object_count = len(r_2["class_ids"])
    for i in range(object_count):      
        mask_2=r_2["masks"][:, :, i]
        contours_2 = get_mask_contours(mask_2)
        for cnt in contours_2:
            cv2.fillPoly(img_zeros, [cnt],255)#only the outlines
        
            
    _,buffer=cv2.imencode('.jpg',transparent)
    img_encoded=base64.b64encode(buffer)
    img_st=encode((str(img_encoded)[2:])[:-1])
    
    my_dict=dict()
    my_dict['img_st']=img_st
        
    
    
    _,buffer=cv2.imencode('.jpg',img_zeros)
    img_encoded=base64.b64encode(buffer)
    img_st=encode((str(img_encoded)[2:])[:-1])
    my_dict['img_st_2']=img_st
    yolo_st=get_yolo_encoded(img)
    my_dict['osteophytes']=yolo_st
    K.clear_session()
    return my_dict 
app.run()

VERS 0.2


Using TensorFlow backend.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  mask_rcnn_object_0005.h5


Loading weights from  knee_space_narrowing.h5




127.0.0.1 - - [15/May/2022 22:14:34] "GET /knee?image=... HTTP/1.1" 200 -


Loading weights from  mask_rcnn_object_0005.h5
Loading weights from  knee_space_narrowing.h5


127.0.0.1 - - [15/May/2022 22:14:52] "GET /knee?image=... HTTP/1.1" 200 -


Loading weights from  mask_rcnn_object_0005.h5
Loading weights from  knee_space_narrowing.h5


127.0.0.1 - - [15/May/2022 22:15:09] "GET /knee?image=... HTTP/1.1" 200 -
